# Obtain GOES16/18 SW data for all of 2023

## Open single file

In [1]:
import xarray as xr
import fsspec
import gcsfs
import numpy as np

url = "https://storage.googleapis.com/gcp-public-data-goes-18/ABI-L2-DSRF/2023/227/16/OR_ABI-L2-DSRF-M6_G18_s20232271600219_e20232271609527_c20232271623259.nc"
fp = fsspec.open(url)
ds = xr.open_dataset(fp.open(), engine="h5netcdf")

In [2]:
ds

<xarray.Dataset>
Dimensions:                                 (lat: 326, lon: 326,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2,
                                             number_of_LZA_bounds: 2,
                                             number_of_SZA_bounds: 2,
                                             number_of_wavelength_bounds: 2)
Coordinates:
    t                                       datetime64[ns] ...
  * lat                                     (lat) float64 81.25 80.75 ... -81.25
  * lon                                     (lon) float64 -218.2 ... -55.75
    lat_image                               float32 ...
    lon_image                               float32 ...
    retrieval_local_zenith_angle            float32 ...
    quantitative_local_zenith_angle         float32 ...
    retrieval_solar_zenith_angle            float32 ...
    quantitative_solar_zenith_angle         float32 ...
    dsr_product_wavelength                  float32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                number_of_LZA_bounds, number_of_SZA_bounds,
                                number_of_wavelength_bounds
Data variables: (12/32)
    DSR                                     (lat, lon) float32 ...
    DQF                                     (lat, lon) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_lat_lon_projection                 int32 ...
    lat_image_bounds                        (number_of_image_bounds) float32 ...
    lon_image_bounds                        (number_of_image_bounds) float32 ...
    ...                                      ...
    std_dev_dsr                             float32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    algorithm_dynamic_input_data_container  int32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/29)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    cdm_data_type:             Image
    time_coverage_start:       2023-08-15T16:00:21.9Z
    time_coverage_end:         2023-08-15T16:09:52.7Z
    timeline_id:               ABI Mode 6
    production_data_source:    Realtime
    id:                        92103fc7-1d41-431c-a12e-636c8a40c3c8

## Open all files in `2023` dir using gcsfs.GCSFilesystem route

In [32]:
#make a list of second to last dir (ending with the day #)
day_dir_list = [("gcp-public-data-goes-16/ABI-L2-DSRF/2023/" + f"{i}/") for i in ["{:03d}".format(i) for i in np.arange(1,366)]]

In [33]:
day_dir_list

['gcp-public-data-goes-16/ABI-L2-DSRF/2023/001/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/002/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/003/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/004/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/005/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/006/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/007/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/008/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/009/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/010/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/011/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/012/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/013/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/014/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/015/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/016/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/017/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/018/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/019/',
 'gcp-public-data-goes-16/ABI-L2-DSRF/2023/020/',


In [11]:
#flatten file_list
import itertools

def flatten_list(nested_list):
    return list(itertools.chain(*nested_list))

In [39]:
#make a list of all the hourly dirs
hour_list = []

for i in np.arange(365):
    hour_list.append(gcsfs.GCSFileSystem().ls(day_dir_list[i]))

#make a flattened list of all the files (hourly)
flattened_hour_list = flatten_list(hour_list)
file_list = []

for i in np.arange(len(flattened_hour_list)):
    file_list.append(gcsfs.GCSFileSystem().ls(flattened_hour_list[i]))

In [43]:
file_list[0][0]

'gcp-public-data-goes-16/ABI-L2-DSRF/2023/001/00/OR_ABI-L2-DSRF-M6_G16_s20230010000206_e20230010009514_c20230010021164.nc'

In [44]:
#add http protocol to each dir path
url_list = []
for i in np.arange(8728):
    url_list.append("https://storage.googleapis.com/" + file_list[i][0])

In [46]:
url_list[0]

'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-DSRF/2023/001/00/OR_ABI-L2-DSRF-M6_G16_s20230010000206_e20230010009514_c20230010021164.nc'

In [76]:
fp_list = list(fps)

In [79]:
fp_list[0].open()

<File-like object HTTPFileSystem, https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-DSRF/2023/001/00/OR_ABI-L2-DSRF-M6_G16_s20230010000206_e20230010009514_c20230010021164.nc>

In [84]:
ds = xr.open_mfdataset([fp_list[i].open() for i in np.arange(8728)], engine='h5netcdf')

KeyboardInterrupt: 

## [AWS GOES-2-GO package](https://github.com/blaylockbk/goes2go) route